In [1]:
# Cell 1: 환경 설정 및 필요한 라이브러리 import
import torch
import torch.nn as nn
import numpy as np
import random
import os

# 재현성을 위한 random seed 설정
seed = 20250318
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# 패키지 구조에 맞게 모듈 import
from Datasets.dataset import SeqRecDataset, seq_collate_fn, BucketBatchSampler
from models.model import SeqRecModel
from models.sub1_sequence_embedding import sentence_embed, sentence_embedder
from models.sub2_time_gap import TimeGapEmbedding
from models.sub3_attention import MultiHeadSelfAttention
from models.sub4_user_embedding import UserEmbeddingUpdater
from models.sub5_FFN import preprocess_inputs, create_ffn_model

print("환경 및 모듈 임포트 완료!")


/home/jy1559/.conda/envs/first/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


환경 및 모듈 임포트 완료!


In [2]:
# Cell 1.5: 테스트에 사용할 하이퍼파라미터 설정
dataset_name = "Globo"
hyperparams = {
    # 모델 관련 하이퍼파라미터
    "hf_model_path": "sentence-transformers/all-MiniLM-L6-v2",
    "embed_dim": 384,
    "ffn_hidden_dim": 512,
    "time_gap_hidden_dim": 128,
    "num_attention_heads": 8,
    "dropout": 0.2,
    "strategy": "EachSession_LastInter",
    "update_flags": {
         "llm": True,
         "tg": True,
         "attention": True,
         "ffn": True,
         "user_emb": True,
         "init_emb": True
    },
    "lora": {
         "use": True,
         "r": 4,
         "alpha": 32
    },
    # 데이터 경로 및 배치 사이즈
    "dataset_paths": {
         "interactions_path": f"./Datasets/{dataset_name}/interactions.json",
         "item_metadata_path": f"./Datasets/{dataset_name}/item_metadata.json"
    },
    "batch_size": 2,  # 원하는 배치 사이즈로 변경 가능
    "use_bucket_batching": True  # True이면 BucketBatchSampler 사용
}

print("하이퍼파라미터 설정 완료!")
print(hyperparams)


하이퍼파라미터 설정 완료!
{'hf_model_path': 'sentence-transformers/all-MiniLM-L6-v2', 'embed_dim': 384, 'ffn_hidden_dim': 512, 'time_gap_hidden_dim': 128, 'num_attention_heads': 8, 'dropout': 0.2, 'strategy': 'EachSession_LastInter', 'update_flags': {'llm': True, 'tg': True, 'attention': True, 'ffn': True, 'user_emb': True, 'init_emb': True}, 'lora': {'use': True, 'r': 4, 'alpha': 32}, 'dataset_paths': {'interactions_path': './Datasets/Globo/interactions.json', 'item_metadata_path': './Datasets/Globo/item_metadata.json'}, 'batch_size': 2, 'use_bucket_batching': True}


In [3]:
# Cell 2: 데이터셋 로드 및 DataLoader 구성 (hyperparams["use_bucket_batching"]에 따라 결정)
from torch.utils.data import DataLoader
from Datasets.dataset import SeqRecDataset, seq_collate_fn, BucketBatchSampler

# hyperparams에서 경로 가져오기
interactions_path = hyperparams["dataset_paths"]["interactions_path"]
item_metadata_path = hyperparams["dataset_paths"]["item_metadata_path"]

dataset = SeqRecDataset(interactions_path, item_metadata_path)

if hyperparams.get("use_bucket_batching", False):
    # BucketBatchSampler를 사용하여 DataLoader 구성
    sampler = BucketBatchSampler(dataset, batch_size=hyperparams["batch_size"], shuffle_batches=True)
    dataloader = DataLoader(dataset, batch_sampler=sampler, collate_fn=seq_collate_fn)
else:
    # 기본 DataLoader 사용
    dataloader = DataLoader(dataset, batch_size=hyperparams["batch_size"], collate_fn=seq_collate_fn)

# 첫 번째 배치 가져오기
batch_data = next(iter(dataloader))

# 배치 딕셔너리의 각 key와 텐서 shape 확인
print("==== DataLoader 배치 확인 ====")
for key, value in batch_data.items():
    if torch.is_tensor(value):
        print(f"{key}: {value.shape}")
    else:
        print(f"{key}")#: {value}")

# bucket batching을 사용한 경우, padding 비율 확인 (-1이 padding으로 사용됨)
if hyperparams.get("use_bucket_batching", False):
    item_ids = batch_data['item_id']  # [B, S, I] 텐서
    padding_ratio = (item_ids == -1).float().mean().item()
    print("Bucket Batch의 전체 padding 비율: {:.2%}".format(padding_ratio))


==== DataLoader 배치 확인 ====
item_id: torch.Size([2, 13, 4])
embedding_sentences
delta_ts: torch.Size([2, 13, 4])
interaction_mask: torch.Size([2, 13, 4])
session_mask: torch.Size([2, 13])
Bucket Batch의 전체 padding 비율: 38.46%


In [4]:
# Cell 3: 문장 임베딩 테스트
tokenizer, sentence_model = sentence_embedder(hyperparams["hf_model_path"])
sentences = batch_data['embedding_sentences']
interaction_mask = batch_data['interaction_mask']  # [B, S, I] 텐서

embeddings = sentence_embed(tokenizer, sentence_model, sentences, interaction_mask)
print("문장 임베딩 출력 shape:", embeddings.shape)


문장 임베딩 출력 shape: torch.Size([2, 13, 4, 384])


In [5]:
# Cell 4: 시간 간격 임베딩 테스트
delta_ts = batch_data['delta_ts']
time_gap_embedder = TimeGapEmbedding(
    embedding_dim=hyperparams["embed_dim"],
    hidden_dim=hyperparams["time_gap_hidden_dim"]
)
time_gap_embeddings = time_gap_embedder(delta_ts)
print("시간 간격 임베딩 출력 shape:", time_gap_embeddings.shape)


시간 간격 임베딩 출력 shape: torch.Size([2, 13, 4, 384])


In [6]:
# Cell 5: MultiHeadSelfAttention 테스트
combined_emb = embeddings + time_gap_embeddings  # [B, S, I, D]
attention_mask = interaction_mask

attention_module = MultiHeadSelfAttention(
    embedding_dim=hyperparams["embed_dim"],
    num_heads=hyperparams["num_attention_heads"],
    dropout=hyperparams["dropout"]
)
attention_out = attention_module(combined_emb, attention_mask)
print("Attention 출력 shape:", attention_out.shape)


Attention 출력 shape: torch.Size([2, 13, 4, 384])


In [7]:
# Cell 6: UserEmbeddingUpdater 테스트
user_updater = UserEmbeddingUpdater(embedding_dim=hyperparams["embed_dim"])
session_mask = batch_data['session_mask']

updated_user_emb = user_updater(attention_out, interaction_mask, session_mask)
print("업데이트된 사용자 임베딩 shape:", updated_user_emb.shape)


업데이트된 사용자 임베딩 shape: torch.Size([2, 384])


In [8]:
# Cell 7: FFN 테스트
ffn_input = preprocess_inputs(attention_out, time_gap_embeddings, updated_user_emb)
print("FFN 입력 shape:", ffn_input.shape)

ffn_model = create_ffn_model(
    input_dim=ffn_input.shape[-1],
    hidden_dim=hyperparams["ffn_hidden_dim"],
    output_dim=ffn_input.shape[-1]
)
ffn_out = ffn_model(ffn_input)
print("FFN 출력 shape:", ffn_out.shape)


torch.Size([2, 13, 4, 384])
torch.Size([2, 13, 4, 384])
torch.Size([2, 1, 1, 384])
FFN 입력 shape: torch.Size([2, 13, 4, 384])
FFN 출력 shape: torch.Size([2, 13, 4, 384])


In [9]:
# Cell 8: 전체 모델 (SeqRecModel) 단일 배치 forward 테스트
model = SeqRecModel(
    embed_dim=hyperparams["embed_dim"],
    strategy=hyperparams["strategy"],
    update=hyperparams["update_flags"],
    lora=hyperparams["lora"],
    ffn_hidden_dim=hyperparams["ffn_hidden_dim"],
    time_gap_hidden_dim=hyperparams["time_gap_hidden_dim"],
    num_attention_heads=hyperparams["num_attention_heads"],
    dropout=hyperparams["dropout"],
    hf_model_path=hyperparams["hf_model_path"]
)
model.eval()

with torch.no_grad():
    output_features, new_user_emb = model(batch_data)

print("모델 출력 feature shape:", output_features.shape)
print("업데이트된 사용자 임베딩 shape:", new_user_emb.shape)


torch.Size([2, 13, 4, 384])
torch.Size([2, 13, 4, 384])
torch.Size([2, 1, 1, 384])
모델 출력 feature shape: torch.Size([2, 13, 384])
업데이트된 사용자 임베딩 shape: torch.Size([2, 384])


In [10]:
# Cell 9: 전체 결과 정리
print("==== 최종 확인 ====")
print("문장 임베딩:", embeddings.shape)
print("시간 간격 임베딩:", time_gap_embeddings.shape)
print("Attention 출력:", attention_out.shape)
print("FFN 출력:", ffn_out.shape)
print("모델 최종 출력 feature:", output_features.shape)
print("최종 사용자 임베딩:", new_user_emb.shape)


==== 최종 확인 ====
문장 임베딩: torch.Size([2, 13, 4, 384])
시간 간격 임베딩: torch.Size([2, 13, 4, 384])
Attention 출력: torch.Size([2, 13, 4, 384])
FFN 출력: torch.Size([2, 13, 4, 384])
모델 최종 출력 feature: torch.Size([2, 13, 384])
최종 사용자 임베딩: torch.Size([2, 384])


In [11]:
# Cell 10: 각 모듈별 출력 shape 및 결과 정리
print("==== 최종 확인 ====")
print("문장 임베딩:", embeddings.shape)
print("시간 간격 임베딩:", time_gap_embeddings.shape)
print("Attention 출력:", attention_out.shape)
print("FFN 출력:", ffn_out.shape)
print("모델 최종 출력 feature:", output_features.shape)
print("최종 사용자 임베딩:", new_user_emb.shape)


==== 최종 확인 ====
문장 임베딩: torch.Size([2, 13, 4, 384])
시간 간격 임베딩: torch.Size([2, 13, 4, 384])
Attention 출력: torch.Size([2, 13, 4, 384])
FFN 출력: torch.Size([2, 13, 4, 384])
모델 최종 출력 feature: torch.Size([2, 13, 384])
최종 사용자 임베딩: torch.Size([2, 384])
